In [2]:
ROOT = 'datasets'
DATASET = 'coco'
ANNOTATIONS_PATH = 'annotations/captions_{0}2014.json'
IMAGES_PATH = 'images/{0}2014'

In [3]:
import os

In [8]:
import torchvision.datasets as dset
import torchvision.transforms as transforms

In [7]:
transform = transforms.Compose(
    [transforms.Resize((200, 200)),
    transforms.ToTensor()
     ])

cap = dset.CocoCaptions(root = os.path.join(ROOT, DATASET, IMAGES_PATH.format('train')),
                        annFile = os.path.join(ROOT, DATASET, ANNOTATIONS_PATH.format('train')),
                        transform = transform)

print('Number of samples: ', len(cap))

loading annotations into memory...
Done (t=0.55s)
creating index...
index created!
Number of samples:  82783


In [40]:
from IPython.display import display

display(cap[0][0])
print(cap[0][1])

tensor([[[0.3216, 0.3098, 0.3333,  ..., 0.4627, 0.5294, 0.7647],
         [0.3216, 0.2980, 0.3373,  ..., 0.4745, 0.5451, 0.7490],
         [0.3137, 0.3020, 0.3216,  ..., 0.4706, 0.4824, 0.5176],
         ...,
         [0.1922, 0.1647, 0.1490,  ..., 0.6235, 0.6118, 0.5961],
         [0.1922, 0.1843, 0.1765,  ..., 0.6078, 0.6039, 0.6039],
         [0.2471, 0.2431, 0.2275,  ..., 0.6000, 0.5882, 0.6000]],

        [[0.1686, 0.1765, 0.1804,  ..., 0.3529, 0.4118, 0.5765],
         [0.1490, 0.1647, 0.1490,  ..., 0.3686, 0.4353, 0.5922],
         [0.1529, 0.1569, 0.1451,  ..., 0.3608, 0.3843, 0.4118],
         ...,
         [0.0902, 0.0706, 0.0627,  ..., 0.6588, 0.6471, 0.6431],
         [0.0706, 0.0745, 0.0824,  ..., 0.6471, 0.6392, 0.6353],
         [0.1333, 0.1333, 0.1216,  ..., 0.6353, 0.6235, 0.6196]],

        [[0.1020, 0.1059, 0.1176,  ..., 0.3647, 0.4039, 0.5294],
         [0.0902, 0.0863, 0.0902,  ..., 0.3686, 0.4275, 0.5569],
         [0.0980, 0.1137, 0.0863,  ..., 0.3490, 0.3765, 0.

['A restaurant has modern wooden tables and chairs.', 'A long restaurant table with rattan rounded back chairs.', 'a long table with a plant on top of it surrounded with wooden chairs ', 'A long table with a flower arrangement in the middle for meetings', 'A table is adorned with wooden chairs with blue accents.']


# Create mini-dataset

In [1]:
import json

In [97]:
with open(os.path.join(ROOT, DATASET, ANNOTATIONS_PATH.format('train'))) as f:
    d = json.load(f)

In [99]:
image_ids = []
d['images'] = d['images'][:5]
for image in d['images']:
    image_ids.append(image['id'])

In [110]:
annotations = []
for annotation in d['annotations']:
    if annotation['image_id'] in image_ids:
        annotations.append(annotation)
d['annotations'] = annotations

In [111]:
os.mkdir(os.path.join(ROOT, 'mini_coco'))

In [113]:
image_filenames = []
for image in d['images']:
    image_filenames.append(image['file_name'])

In [114]:
os.mkdir(os.path.join(ROOT, 'mini_coco', 'images'))
os.mkdir(os.path.join(ROOT, 'mini_coco', 'annotations'))

In [117]:
os.mkdir(os.path.join(ROOT, 'mini_coco', IMAGES_PATH.format('train')))

FileExistsError: [Errno 17] File exists: 'datasets/mini_coco/images/train2014'

In [118]:
from shutil import copyfile

for filename in image_filenames:
    copyfile(os.path.join(ROOT, DATASET, IMAGES_PATH.format('train'), filename),
             os.path.join(ROOT, 'mini_coco', IMAGES_PATH.format('train'), filename))

In [123]:
with open(os.path.join(ROOT, 'mini_coco', ANNOTATIONS_PATH.format('train')), 'w') as f:
    json.dump(d, f)

# Prepare dictionary

In [172]:
cap = dset.CocoCaptions(root = os.path.join(ROOT, 'mini_coco', IMAGES_PATH.format('train')),
                        annFile = os.path.join(ROOT, 'mini_coco', ANNOTATIONS_PATH.format('train')))

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [173]:
from nltk.tokenize import word_tokenize
import string
from collections import defaultdict

c = defaultdict(int)

for image, texts in cap:
    for text in texts:
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = text.lower()
        text = word_tokenize(text)
        for word in text:
            c[word] += 1

In [174]:
c_filtered = [word for word in c if c[word] > 3]

In [176]:
START = '<START>'
UNK = '<UNK>'
END = '<END>'

c_filtered.append(START)
c_filtered.append(UNK)
c_filtered.append(END)

In [177]:
i2w = {}
w2i = {}

for index, word in enumerate(c_filtered):
    i2w[index] = word
    w2i[word] = index

In [178]:
print(i2w)
print(w2i)

{0: 'a', 1: 'and', 2: 'chairs', 3: 'table', 4: 'with', 5: 'on', 6: 'of', 7: 'in', 8: 'the', 9: 'is', 10: 'man', 11: 'kitchen', 12: 'small', 13: 'person', 14: 'booth', 15: 'phone', 16: '<START>', 17: '<UNK>', 18: '<END>'}
{'a': 0, 'and': 1, 'chairs': 2, 'table': 3, 'with': 4, 'on': 5, 'of': 6, 'in': 7, 'the': 8, 'is': 9, 'man': 10, 'kitchen': 11, 'small': 12, 'person': 13, 'booth': 14, 'phone': 15, '<START>': 16, '<UNK>': 17, '<END>': 18}


# Transform text to index sequence

In [179]:
def transform_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    text = word_tokenize(text)
    
    sequence = [w2i[START]]
    for word in text:
        if word in w2i:
            sequence.append(w2i[word])
        else:
            sequence.append(w2i[UNK])
    sequence.append(w2i[END])
    return sequence

In [180]:
transform_text('A pen with, a, kitchen!!.')

[16, 0, 17, 4, 0, 11, 18]

# Train model

In [20]:
import torch

In [66]:
def collate_fn(batch):
    print(len(batch))
    return batch

In [67]:
trainloader = torch.utils.data.DataLoader(cap, batch_size=4, shuffle=True, collate_fn=collate_fn)

In [68]:
b = None

for batch in trainloader:
    b = batch
    break

4


In [46]:
cap[0][0].size()

torch.Size([3, 200, 200])

In [47]:
cap[1][0].size()

torch.Size([3, 200, 200])

In [64]:
b[1]

[('A white bus is parked on a road near a small hill and the ocean.',
  'Grey storm clouds loom over a city street in a business district.',
  'A train car with four beds next to a window.',
  'a silver and black train engine and some people '),
 ('A van parked at the side of a graveled road.',
  'An empty sign stands near a city street.',
  'a room with two bunk beds in it next to a big window ',
  'A man showing some visitors a steam engine.'),
 ('A bus sitting next to a body of water on a gravel road.',
  'A sign that is standing in a parking lot.',
  'Beds are in a small room near a window and table with bottles on it. ',
  'An old fashion train engine is being observed by a group of people. '),
 ('A van is pulled up to a boat docking area while a cow stands alongside the signs.',
  'Empty commercial building on a city street corner.',
  'Two beers sit on a table between bunk beds.',
  'A train that is on some train tracks.'),
 ('There is a can pulled over on the side of the road n

In [23]:
import torch
from torch import nn

In [20]:
embed = nn.Embedding(18, 5)

In [27]:
inp = torch.LongTensor([[1, 2, 3, 4, 5], [5, 6]])

ValueError: expected sequence of length 5 at dim 1 (got 2)

In [26]:
embed(inp).shape

torch.Size([5, 5])

In [34]:
from torch.nn.utils.rnn import pack_sequence
a = torch.tensor([1,2,3])
b = torch.tensor([4,5])
c = torch.tensor([6])
pack_sequence([a, b, c])

PackedSequence(data=tensor([1, 4, 6, 2, 5, 3]), batch_sizes=tensor([3, 2, 1]), sorted_indices=None, unsorted_indices=None)

In [33]:
packed.data = embed(packed.data).shape

AttributeError: can't set attribute

In [35]:
nn.utils.rnn.PackedSequence(
    a, torch.tensor([1, 2]))

PackedSequence(data=tensor([1, 2, 3]), batch_sizes=tensor([1, 2]), sorted_indices=None, unsorted_indices=None)

In [159]:
class SimpleModel(nn.Module):
    def __init__(self, *args, **kwargs):
        super(SimpleModel, self).__init__(*args, **kwargs)
        self.encoder = nn.Conv2d(3, 5, 3)
        self.embeddings = nn.Embedding(18, 7)
        self.decoder = nn.RNN(7, 3)
        self.linear = nn.Linear(196020, 3)
        self.last_linear = nn.Linear(3, 18)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, image, captions):
        h0 = self.linear(self.encoder(image).view(-1, 196020))
        h0 = h0.repeat(1, 5, 1)
        packed_embeds = nn.utils.rnn.PackedSequence(self.embeddings(captions.data), captions.batch_sizes)
        decoded, _ = self.decoder(packed_embeds, h0)
        probs = self.softmax(self.last_linear(decoded.data))
        return nn.utils.rnn.PackedSequence(probs, decoded.batch_sizes)

In [55]:
def collate_fn(batch):
    return batch

trainloader = torch.utils.data.DataLoader(cap, batch_size=1, shuffle=False, collate_fn=collate_fn)

In [56]:
sample_batch = None

for b in trainloader:
    sample_batch = b
    break

In [57]:
sample_batch

[(<PIL.Image.Image image mode=RGB size=640x480 at 0x7F7125F94978>,
  ['A restaurant has modern wooden tables and chairs.',
   'A long restaurant table with rattan rounded back chairs.',
   'a long table with a plant on top of it surrounded with wooden chairs ',
   'A long table with a flower arrangement in the middle for meetings',
   'A table is adorned with wooden chairs with blue accents.'])]

In [81]:
transform = transforms.Compose(
    [transforms.Resize((200, 200)),
    transforms.ToTensor()
     ])

image = transform(sample_batch[0][0])

In [96]:
image = torch.unsqueeze(image, 0)

In [64]:
transformed_texts = []

for text in sample_batch[0][1]:
    transformed_texts.append(torch.tensor(transform_text(text)))

In [65]:
transformed_texts

[tensor([ 0, 16, 16, 16, 16, 16,  1,  2, 17]),
 tensor([ 0, 16, 16,  3,  4, 16, 16, 16,  2, 17]),
 tensor([ 0, 16,  3,  4,  0, 16,  5, 16,  6, 16, 16,  4, 16,  2, 17]),
 tensor([ 0, 16,  3,  4,  0, 16, 16,  7,  8, 16, 16, 16, 17]),
 tensor([ 0,  3,  9, 16,  4, 16,  2,  4, 16, 16, 17])]

In [67]:
packed = pack_sequence(transformed_texts, enforce_sorted=False)

In [160]:
model = SimpleModel()
ans = model.forward(image, packed)

In [162]:
ans[0].data.shape

torch.Size([58, 18])

In [165]:
output, input_sizes = torch.nn.utils.rnn.pad_packed_sequence(ans)

In [167]:
input_sizes

tensor([15, 13, 11, 10,  9])

In [168]:
transformed_texts

[tensor([ 0, 16, 16, 16, 16, 16,  1,  2, 17]),
 tensor([ 0, 16, 16,  3,  4, 16, 16, 16,  2, 17]),
 tensor([ 0, 16,  3,  4,  0, 16,  5, 16,  6, 16, 16,  4, 16,  2, 17]),
 tensor([ 0, 16,  3,  4,  0, 16, 16,  7,  8, 16, 16, 16, 17]),
 tensor([ 0,  3,  9, 16,  4, 16,  2,  4, 16, 16, 17])]

In [169]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()

In [170]:
input.shape

torch.Size([3, 5])

In [171]:
target.shape

torch.Size([3])

In [ ]:
loss = nn.CrossEntropyLoss()

In [183]:
ans[0].shape

torch.Size([58, 18])

In [184]:
packed.data.shape

torch.Size([58])

In [185]:
loss(ans[0], packed.data)

tensor(2.9096, grad_fn=<NllLossBackward>)

In [186]:
model.parameters()

<generator object Module.parameters at 0x7f7125e7c3b8>